In [1]:
from transformers import AutoTokenizer, GemmaTokenizer
import pandas as pd
from tqdm import tqdm

/home/administrator/miniconda3/envs/vllm/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Текст для подсчета токенов
text = """Гляциологи из Калифорнийского университета в Ирвайне обнаружили доказательства проникновения теплой морской воды под антарктический ледник Туэйтса, прозванный ледником Судного дня. Согласно результатам исследования, опубликованного в журнале Труды Национальной академии наук, это указывает на то, что тревожные изменения, происходящие с ледником, могут стать необратимыми.
Ледник Туэйтса располагается в Западной Антарктиде и частично является шельфовым, то есть его часть опирается на морское дно. Предполагается, что таяние ледника вызовет сход в океан огромной массы льда, располагающегося на суше, что вызовет резкий подъем уровня моря. Изменение климата способствует вторжению теплых вод у морского дна, которые воздействуют на линию заземления, где ледник отрывается от морского дна, и создают протяженные полости между дном и телом ледника. Таким образом, таяние усиливается и может приобрести необратимый характер.
В новой работе ученые проанализировали данные, собранные с марта по июнь 2023 года созвездием коммерческих спутников, которые вращаются вокруг Земли по полярной орбите (пролегающей под прямым углом к экватору). На их борту находится интерферометрический радар с синтезированной апертурой, для постоянного мониторинга изменений на поверхности Земли. Спутники предоставили гляциологам долгосрочную серию ежедневных наблюдений за ледником Туэйтса, что позволило сопоставить происходящие с ним изменения с циклом приливов и отливов.
Оказалось, что приливы вызывают периодическое отступление линии заземления на 2-6 километра. Морская вода, которая с приливом нагнетается в полости у подножия ледника, смешивается с талой пресной водой и создает достаточное давление, чтобы поднять лед над дном. В результате образуется еще больше пространства, куда может проникать теплая вода. Подсчитано, что нерегулярные вторжения морской воды происходят еще на шесть километров вглубь ледника. Поскольку морская вода замерзает при минус двух градусах Цельсия, она в большей степени способствует таянию Туэйтса, чем пресная вода.
Авторы отмечают, что на данный момент недостаточно информации, чтобы с уверенностью сказать, сколько времени осталось до того, как вторжение океанской воды станет необратимым."""

# Подсчет количества символов и слов
char_count = len(text)
word_count = len(text.split())

In [15]:
# Список имен моделей
model_names = ["Vikhrmodels/Vikhr-7B-instruct_0.4", 
               "unsloth/Mistral-Nemo-Instruct-2407",
               "NousResearch/Meta-Llama-3-8B-Instruct", 
               "openchat/openchat-3.5-0106", 
               "microsoft/Phi-3-medium-128k-instruct",
               "Xenova/gpt-4o",
               "Qwen/Qwen2-7B-Instruct",
               "unsloth/gemma-2-9b",
               "deepseek-ai/DeepSeek-V2-Lite-Chat",
               "AnatoliiPotapov/T-lite-instruct-0.1",
               "CohereForAI/aya-23-35B"]

# Список для хранения результатов
results = []

# Первая модель для сравнения
first_model = model_names[0]
first_model_token_count = len(AutoTokenizer.from_pretrained(first_model).encode(text))

# Для каждой модели
for model_name in tqdm(model_names):
    # Загрузка модели и токенизатора
    tokenizer = AutoTokenizer.from_pretrained(model_name)

    # Подсчет токенов
    token_count = len(tokenizer.tokenize(text))

    # Изменение в процентах
    percent_change = (token_count - first_model_token_count) / first_model_token_count * 100

    # Размер словаря токенизатора
    vocab_size = len(tokenizer)

    # Добавление результата в список
    results.append({'model_name': model_name,
                    'ru_token_count': token_count,
                    'percent_change': int(percent_change),
                    'char_per_token': round(char_count / token_count, 1),
                    'word_per_token': round(word_count / token_count, 1),
                    'vocab_size': vocab_size})  

df = pd.DataFrame(results)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
100%|██████████| 11/11 [00:05<00:00,  1.93it/s]


In [10]:
print(text)

Гляциологи из Калифорнийского университета в Ирвайне обнаружили доказательства проникновения теплой морской воды под антарктический ледник Туэйтса, прозванный ледником Судного дня. Согласно результатам исследования, опубликованного в журнале Труды Национальной академии наук, это указывает на то, что тревожные изменения, происходящие с ледником, могут стать необратимыми.
Ледник Туэйтса располагается в Западной Антарктиде и частично является шельфовым, то есть его часть опирается на морское дно. Предполагается, что таяние ледника вызовет сход в океан огромной массы льда, располагающегося на суше, что вызовет резкий подъем уровня моря. Изменение климата способствует вторжению теплых вод у морского дна, которые воздействуют на линию заземления, где ледник отрывается от морского дна, и создают протяженные полости между дном и телом ледника. Таким образом, таяние усиливается и может приобрести необратимый характер.
В новой работе ученые проанализировали данные, собранные с марта по июнь 2023

In [11]:
# Вывод количества символов и слов
print(f"Количество символов: {char_count}")
print(f"Количество слов: {word_count}")

Количество символов: 2210
Количество слов: 288


In [17]:
pd.set_option('display.width', 1000)
print(df.sort_values('ru_token_count').reset_index(drop=True))

                               model_name  ru_token_count  percent_change  char_per_token  word_per_token  vocab_size
0       Vikhrmodels/Vikhr-7B-instruct_0.4             462               0             4.8             0.6       79099
1                  CohereForAI/aya-23-35B             533              15             4.1             0.5      255029
2                           Xenova/gpt-4o             585              26             3.8             0.5      200000
3                      unsloth/gemma-2-9b             605              30             3.7             0.5      256000
4      unsloth/Mistral-Nemo-Instruct-2407             633              37             3.5             0.5      131072
5   NousResearch/Meta-Llama-3-8B-Instruct             677              46             3.3             0.4      128256
6     AnatoliiPotapov/T-lite-instruct-0.1             680              47             3.2             0.4      128259
7                  Qwen/Qwen2-7B-Instruct             75